In [1]:
import time

import numpy as np
import pandas as pd
import torch
from torch import nn
from tqdm import tqdm

# random seed
np.random.seed(42)
torch.manual_seed(42)

In [2]:
# turn off warning
import logging
logging.getLogger('pyomo.core').setLevel(logging.ERROR)

## Problem Setting

In [3]:
# init
num_data = 5000   # number of data
num_vars = 5      # number of decision variables
num_ints = 5      # number of integer decision variables
test_size = 1000  # number of test size
val_size = 1000   # number of validation size
train_size = num_data - test_size - val_size

In [4]:
# parameters as input data
p_train = np.random.uniform(1, 11, (train_size, num_vars)).astype(np.float32)
p_test = np.random.uniform(1, 11, (test_size, num_vars)).astype(np.float32)
p_dev = np.random.uniform(1, 11, (val_size, num_vars)).astype(np.float32)

In [5]:
# nm datasets
from neuromancer.dataset import DictDataset
data_train = DictDataset({"p":p_train}, name="train")
data_test = DictDataset({"p":p_test}, name="test")
data_dev = DictDataset({"p":p_dev}, name="dev")

In [6]:
# torch dataloaders
from torch.utils.data import DataLoader
loader_train = DataLoader(data_train, batch_size=32, num_workers=0, collate_fn=data_train.collate_fn, shuffle=True)
loader_test = DataLoader(data_test, batch_size=32, num_workers=0, collate_fn=data_test.collate_fn, shuffle=False)
loader_dev = DataLoader(data_dev, batch_size=32, num_workers=0, collate_fn=data_dev.collate_fn, shuffle=True)

## NM Problem

In [7]:
import neuromancer as nm
from problem.neural import probQuadratic

def getNMProb(round_module):
    # parameters
    p = nm.constraint.variable("p")
    # variables
    x_bar = nm.constraint.variable("x_bar")
    x_rnd = nm.constraint.variable("x_rnd")

    # model
    obj_bar, constrs_bar = probQuadratic(x_bar, p, num_vars=num_vars, alpha=100)
    obj_rnd, constrs_rnd = probQuadratic(x_rnd, p, num_vars=num_vars, alpha=100)

    # define neural architecture for the solution mapping
    func = nm.modules.blocks.MLP(insize=num_vars, outsize=num_vars, bias=True,
                                 linear_map=nm.slim.maps["linear"], nonlin=nn.ReLU, hsizes=[80]*4)
    # solution map from model parameters: sol_map(p) -> x
    sol_map = nm.system.Node(func, ["p"], ["x_bar"], name="smap")

    # penalty loss for mapping
    components = [sol_map]
    loss = nm.loss.PenaltyLoss(obj_bar, constrs_bar)
    problem = nm.problem.Problem(components, loss)

    # penalty loss for rounding
    components = [sol_map, round_module]
    loss = nm.loss.PenaltyLoss(obj_rnd, constrs_rnd)
    problem_rnd = nm.problem.Problem(components, loss)

    return problem, problem_rnd

## Exact Solver

In [8]:
from problem.solver import exactQuadratic
model = exactQuadratic(n_vars=num_vars, n_integers=num_ints)

In [9]:
objvals, conviols, elapseds = [], [], []
for p in tqdm(p_test):
    model.setParamValue(*p)
    tick = time.time()
    xval, objval = model.solve("scip")
    tock = time.time()
    objvals.append(objval)
    conviols.append(sum(model.calViolation()))
    elapseds.append(tock - tick)
df = pd.DataFrame({"Obj Val": objvals, "Constraints Viol": conviols, "Elapsed Time": elapseds})
time.sleep(1)
print(df.describe())

100%|██████████████████████████████████████████████████████████████████████████████| 1000/1000 [01:35<00:00, 10.49it/s]


           Obj Val  Constraints Viol  Elapsed Time
count  1000.000000       1000.000000   1000.000000
mean    200.432000          3.835546      0.094654
std      78.224654          5.574005      0.042749
min      36.000000          0.000000      0.061455
25%     149.000000          0.000000      0.076598
50%     190.000000          0.000000      0.077751
75%     247.000000          8.126751      0.091685
max     511.000000         25.352531      0.423046


## Heuristic

In [10]:
from heuristic import naive_round

In [11]:
# relaxed model
model_rel = model.relax()

In [12]:
sols, objvals, conviols, elapseds = [], [], [], []
for p in tqdm(p_test):
    model_rel.setParamValue(*p)
    tick = time.time()
    xval_init, _ = model_rel.solve("scip", max_iter=100)
    naive_round(xval_init, model)
    tock = time.time()
    xval, objval = model.getVal()
    sols.append(xval.values())
    objvals.append(objval)
    conviols.append(sum(model.calViolation()))
    elapseds.append(tock - tick)
df = pd.DataFrame({"Sol":sols, "Obj Val": objvals, "Constraints Viol": conviols, "Elapsed Time": elapseds})
time.sleep(1)
print(df.describe())

100%|██████████████████████████████████████████████████████████████████████████████| 1000/1000 [02:12<00:00,  7.57it/s]


          Obj Val  Constraints Viol  Elapsed Time
count  1000.00000       1000.000000   1000.000000
mean    179.96800         28.608444      0.131204
std      76.37923          7.851483      0.051891
min      26.00000         10.634741      0.074227
25%     130.00000         21.912093      0.105940
50%     170.00000         27.912093      0.124206
75%     222.00000         33.912093      0.140023
max     471.00000         53.912093      0.686334


## Learning to Round

In [13]:
from model.layer import netFC
from model.round import roundModel
# round x
layers_rnd = netFC(input_dim=num_vars*2, hidden_dims=[80]*4, output_dim=num_vars)
round_func = roundModel(layers=layers_rnd, param_keys=["p"], var_keys=["x_bar"], output_keys=["x_rnd"],
                        int_ind={"x_bar":model.intInd}, name="round")
_, problem = getNMProb(round_func)

In [14]:
# training
lr = 0.001    # step size for gradient descent
epochs = 400  # number of training epochs
warmup = 50   # number of epochs to wait before enacting early stopping policy
patience = 50 # number of epochs with no improvement in eval metric to allow before early stopping
# set adamW as optimizer
optimizer = torch.optim.AdamW(problem.parameters(), lr=lr)
# define trainer
trainer = nm.trainer.Trainer(problem, loader_train, loader_dev, loader_test,
                             optimizer, epochs=epochs, patience=patience, warmup=warmup)
best_model = trainer.train()

epoch: 0  train_loss: 937.0948486328125
epoch: 1  train_loss: 402.47930908203125
epoch: 2  train_loss: 381.07696533203125
epoch: 3  train_loss: 372.87158203125
epoch: 4  train_loss: 380.60809326171875
epoch: 5  train_loss: 380.5409240722656
epoch: 6  train_loss: 382.86431884765625
epoch: 7  train_loss: 377.9921875
epoch: 8  train_loss: 372.2610168457031
epoch: 9  train_loss: 370.8432922363281
epoch: 10  train_loss: 370.6485595703125
epoch: 11  train_loss: 361.46246337890625
epoch: 12  train_loss: 358.6501770019531
epoch: 13  train_loss: 364.6545104980469
epoch: 14  train_loss: 356.6761474609375
epoch: 15  train_loss: 355.5833740234375
epoch: 16  train_loss: 351.9570007324219
epoch: 17  train_loss: 357.2872009277344
epoch: 18  train_loss: 349.68865966796875
epoch: 19  train_loss: 350.5256042480469
epoch: 20  train_loss: 351.1717529296875
epoch: 21  train_loss: 346.0594482421875
epoch: 22  train_loss: 348.74774169921875
epoch: 23  train_loss: 348.5304260253906
epoch: 24  train_loss: 346.

In [15]:
sols, objvals, conviols, elapseds = [], [], [], []
for p in tqdm(p_test):
    datapoints = {"p": torch.tensor(np.array([p]), dtype=torch.float32), "name": "test"}
    tick = time.time()
    output = problem(datapoints)
    tock = time.time()
    x = output["test_x_rnd"]
    # get values
    for ind in model.x:
        model.x[ind].value = x[0, ind].item()
    xval, objval = model.getVal()
    sols.append(xval.values())
    objvals.append(objval)
    conviols.append(sum(model.calViolation()))
    elapseds.append(tock - tick)
df = pd.DataFrame({"Sol":sols, "Obj Val": objvals, "Constraints Viol": conviols, "Elapsed Time": elapseds})
time.sleep(1)
print(df.describe())

100%|█████████████████████████████████████████████████████████████████████████████| 1000/1000 [00:03<00:00, 255.30it/s]


           Obj Val  Constraints Viol  Elapsed Time
count  1000.000000       1000.000000   1000.000000
mean    201.908000          8.299482      0.003555
std      68.289846          2.787089      0.000657
min      47.000000          2.687708      0.001997
25%     153.000000          6.265262      0.003000
50%     194.000000          8.012799      0.003509
75%     247.000000         10.265262      0.004000
max     471.000000         18.542614      0.007084


## Learnable Threshold

In [16]:
from model.layer import netFC
from model.threshold import roundThresholdModel
# round x
layers_rnd = netFC(input_dim=num_vars*2, hidden_dims=[80]*4, output_dim=num_vars)
round_func = roundThresholdModel(layers=layers_rnd, param_keys=["p"], var_keys=["x_bar"], output_keys=["x_rnd"],
                                 int_ind={"x_bar":model.intInd}, name="round")
_, problem = getNMProb(round_func)

In [17]:
# training
lr = 0.001    # step size for gradient descent
epochs = 400  # number of training epochs
warmup = 50   # number of epochs to wait before enacting early stopping policy
patience = 50 # number of epochs with no improvement in eval metric to allow before early stopping
# set adamW as optimizer
optimizer = torch.optim.AdamW(problem.parameters(), lr=lr)
# define trainer
trainer = nm.trainer.Trainer(problem, loader_train, loader_dev, loader_test,
                             optimizer, epochs=epochs, patience=patience, warmup=warmup)
best_model = trainer.train()

epoch: 0  train_loss: 975.09033203125
epoch: 1  train_loss: 392.9432067871094
epoch: 2  train_loss: 355.19049072265625
epoch: 3  train_loss: 352.5472106933594
epoch: 4  train_loss: 347.5971984863281
epoch: 5  train_loss: 347.3907165527344
epoch: 6  train_loss: 346.7637634277344
epoch: 7  train_loss: 340.3681945800781
epoch: 8  train_loss: 342.6441955566406
epoch: 9  train_loss: 342.302978515625
epoch: 10  train_loss: 339.9844055175781
epoch: 11  train_loss: 340.7442321777344
epoch: 12  train_loss: 339.2669677734375
epoch: 13  train_loss: 339.19537353515625
epoch: 14  train_loss: 338.64849853515625
epoch: 15  train_loss: 337.8860168457031
epoch: 16  train_loss: 338.0227355957031
epoch: 17  train_loss: 337.99822998046875
epoch: 18  train_loss: 340.48162841796875
epoch: 19  train_loss: 337.3781433105469
epoch: 20  train_loss: 337.25927734375
epoch: 21  train_loss: 335.99505615234375
epoch: 22  train_loss: 337.12945556640625
epoch: 23  train_loss: 338.3099670410156
epoch: 24  train_loss: 3

In [18]:
sols, objvals, conviols, elapseds = [], [], [], []
for p in tqdm(p_test):
    datapoints = {"p": torch.tensor(np.array([p]), dtype=torch.float32), "name": "test"}
    tick = time.time()
    output = problem(datapoints)
    tock = time.time()
    x = output["test_x_rnd"]
    # get values
    for ind in model.x:
        model.x[ind].value = x[0, ind].item()
    xval, objval = model.getVal()
    sols.append(xval.values())
    objvals.append(objval)
    conviols.append(sum(model.calViolation()))
    elapseds.append(tock - tick)
df = pd.DataFrame({"Sol":sols, "Obj Val": objvals, "Constraints Viol": conviols, "Elapsed Time": elapseds})
time.sleep(1)
print(df.describe())

100%|█████████████████████████████████████████████████████████████████████████████| 1000/1000 [00:04<00:00, 227.89it/s]


           Obj Val  Constraints Viol  Elapsed Time
count  1000.000000       1000.000000   1000.000000
mean    207.073000          8.717267      0.003930
std      73.458327          3.018748      0.000845
min      40.000000          2.650724      0.002000
25%     156.000000          6.302247      0.003311
50%     198.000000          8.302247      0.003997
75%     255.000000         10.542614      0.004472
max     500.000000         20.109414      0.008491


## Learning to Round with Fixed Solution Mapping

In [19]:
from model.layer import netFC
from model.round import roundModel
# round x
layers_rnd = netFC(input_dim=num_vars*2, hidden_dims=[80]*4, output_dim=num_vars)
round_func = roundModel(layers=layers_rnd, param_keys=["p"], var_keys=["x_bar"], output_keys=["x_rnd"],
                        int_ind={"x_bar":model.intInd}, name="round")
problem, problem_rnd = getNMProb(round_func)

In [20]:
# training for mapping
lr = 0.001    # step size for gradient descent
epochs = 400  # number of training epochs
warmup = 50   # number of epochs to wait before enacting early stopping policy
patience = 50 # number of epochs with no improvement in eval metric to allow before early stopping
# set adamW as optimizer
optimizer = torch.optim.AdamW(problem.parameters(), lr=lr)
# define trainer
trainer = nm.trainer.Trainer(problem, loader_train, loader_dev, loader_test,
                             optimizer, epochs=epochs, patience=patience, warmup=warmup)
best_model = trainer.train()

epoch: 0  train_loss: 932.9334106445312
epoch: 1  train_loss: 356.8347473144531
epoch: 2  train_loss: 336.45758056640625
epoch: 3  train_loss: 331.31048583984375
epoch: 4  train_loss: 324.2931213378906
epoch: 5  train_loss: 318.75872802734375
epoch: 6  train_loss: 316.02508544921875
epoch: 7  train_loss: 313.67919921875
epoch: 8  train_loss: 313.3650817871094
epoch: 9  train_loss: 312.2769775390625
epoch: 10  train_loss: 312.3884582519531
epoch: 11  train_loss: 311.3890686035156
epoch: 12  train_loss: 310.96063232421875
epoch: 13  train_loss: 312.8399353027344
epoch: 14  train_loss: 309.7454528808594
epoch: 15  train_loss: 312.87237548828125
epoch: 16  train_loss: 309.1521301269531
epoch: 17  train_loss: 309.9900817871094
epoch: 18  train_loss: 310.71148681640625
epoch: 19  train_loss: 306.4747619628906
epoch: 20  train_loss: 307.1275634765625
epoch: 21  train_loss: 307.9350891113281
epoch: 22  train_loss: 306.3865661621094
epoch: 23  train_loss: 307.3204345703125
epoch: 24  train_loss

In [21]:
# freeze sol mapping
problem.freeze()
# training for rounding
lr = 0.001    # step size for gradient descent
epochs = 400  # number of training epochs
warmup = 50   # number of epochs to wait before enacting early stopping policy
patience = 50 # number of epochs with no improvement in eval metric to allow before early stopping
# set adamW as optimizer
optimizer = torch.optim.AdamW(problem.parameters(), lr=lr)
# define trainer
trainer = nm.trainer.Trainer(problem_rnd, loader_train, loader_dev, loader_test,
                             optimizer, epochs=epochs, patience=patience, warmup=warmup)
best_model = trainer.train()

epoch: 0  train_loss: 395.63311767578125
epoch: 1  train_loss: 395.4594421386719
epoch: 2  train_loss: 395.5500793457031
epoch: 3  train_loss: 394.0802307128906
epoch: 4  train_loss: 395.0129699707031
epoch: 5  train_loss: 396.0818176269531
epoch: 6  train_loss: 395.1732482910156
epoch: 7  train_loss: 395.8547668457031
epoch: 8  train_loss: 395.42840576171875
epoch: 9  train_loss: 395.6715087890625
epoch: 10  train_loss: 395.98187255859375
epoch: 11  train_loss: 395.4700927734375
epoch: 12  train_loss: 395.11761474609375
epoch: 13  train_loss: 397.1585388183594
epoch: 14  train_loss: 396.5810852050781
epoch: 15  train_loss: 394.6093444824219
epoch: 16  train_loss: 394.5400695800781
epoch: 17  train_loss: 394.8847961425781
epoch: 18  train_loss: 395.0920104980469
epoch: 19  train_loss: 396.547119140625
epoch: 20  train_loss: 395.2701110839844
epoch: 21  train_loss: 395.3181457519531
epoch: 22  train_loss: 396.2032165527344
epoch: 23  train_loss: 397.4877014160156
epoch: 24  train_loss: 

In [22]:
sols, objvals, conviols, elapseds = [], [], [], []
for p in tqdm(p_test):
    datapoints = {"p": torch.tensor(np.array([p]), dtype=torch.float32), "name": "test"}
    tick = time.time()
    output = problem_rnd(datapoints)
    tock = time.time()
    x = output["test_x_rnd"]
    # get values
    for ind in model.x:
        model.x[ind].value = x[0, ind].item()
    xval, objval = model.getVal()
    sols.append(xval.values())
    objvals.append(objval)
    conviols.append(sum(model.calViolation()))
    elapseds.append(tock - tick)
df = pd.DataFrame({"Sol":sols, "Obj Val": objvals, "Constraints Viol": conviols, "Elapsed Time": elapseds})
time.sleep(1)
print(df.describe())

100%|█████████████████████████████████████████████████████████████████████████████| 1000/1000 [00:04<00:00, 237.18it/s]


           Obj Val  Constraints Viol  Elapsed Time
count  1000.000000       1000.000000   1000.000000
mean    189.095000          9.041808      0.003829
std      72.065911          2.930551      0.000786
min      27.000000          2.735447      0.001999
25%     136.000000          6.887273      0.003018
50%     183.500000          8.585647      0.003707
75%     234.000000         10.841309      0.004057
max     443.000000         18.579599      0.007525
